# RSA

## Key generation

Prime factorization problem: given $n = p*q$ composite number, it's computationally infeasible to calculate $p$ and $q$.

### Step 1: choosing 2 random primes

$p,q$: random primes (1024+ bits)

### Step 2: computing the modulus $n$

$n=p*q$

### Step 3: calculate euler's totient
$\phi = (p - 1) * (q - 1)$

### Step 4: choose public exponent

It must be coprime with $\phi$ (meaning that $gcd(\phi(n),e)=1$), but it's usually a prime like: $e=65537$

$e=17$

### Step 5: calculate private exponent

$d=e^{-1}\: mod\:\phi(n)$ (d is the modular inverse of: $d \equiv e \: mod \: \phi(n) \rightarrow e*d \equiv 1 \: mod \: \phi(n) $)

## Keys

Public key: ($e$, $n$)

Private key: ($d$, $n$)

## Usage

Encryption: $c=m^e \space mod \space (n)$

Decryption: $m=c^d \space mod \space (n)$

This works because $m=(m^e)^d \space mod \space (n)$ <=> $m=m^{ed} \space mod \space (n)$

Some restrictions: the message should be padded and should not be a 0. This is still not good for encrypting data that is not sufficiently random. So encrypting some text is not recommended.

In [ ]:
from math import gcd

# Step 1: Choose two small primes
p = 61
q = 53

# Step 2: Compute their product
n = p * q  # n = 3233

# Step 3: Compute Euler's totient
phi = (p - 1) * (q - 1) # phi = 3120

# Step 4: Choose public exponent e
e = 17
assert gcd(e, phi) == 1

# Step 5: Compute private exponent d = e⁻¹ mod phi
d = pow(e, -1, phi) # d = 2753

# Final keys
(e, n), (d, n), n.bit_length()

((17, 3233), (2753, 3233), 12)

In [5]:
m = 2455
print('m', m)
# Encrypt the message
c = pow(m, e, n) # c = 2604
print('c', c)
# Decrypt the message
m1 = pow(c, d, n) # m1 = 2455
print('m1', m1)

m 2455
c 2604
m1 2455


In [14]:
# Now with big numbers
from sympy import nextprime
from random import randrange

def generate_keypair(bits):
    # Step 1: Choose two large primes
    p = nextprime(randrange(2**(bits-1), 2**bits))
    q = nextprime(randrange(2**(bits-1), 2**bits))

    # Step 2: Compute modulus
    n = p * q

    # Step 3: Compute Euler's totient
    phi = (p - 1) * (q - 1)

    # Step 4: Choose public exponent e
    e = 65537
    assert gcd(e, phi) == 1

    # Step 5: Compute private exponent d = e⁻¹ mod phi
    d = pow(e, -1, phi)

    return (e, n), (d, n)

pk, sk = generate_keypair(512)
print('pk', pk)
print('sk', sk)

pk (65537, 101288305646811795148587034543089703939703161282351330972146567248817981897752905308681936277836951124439280788225987460131837011016498511046238563608858842677624381152861314950795645912244994236837428811024917232740818208290986605809719776447430196109225253978679700463692513383417708128462521791359286285169)
sk (74801298579429086979820008313639601766600444068305263709369084765484253259522756061993942870607909996974789063725064728060801982028281908481116014466715872179037779254257779447367201388254602854569982432982406903902444190076588240112814143531127078246979568808056205496679600324249884317563803892846332672513, 101288305646811795148587034543089703939703161282351330972146567248817981897752905308681936277836951124439280788225987460131837011016498511046238563608858842677624381152861314950795645912244994236837428811024917232740818208290986605809719776447430196109225253978679700463692513383417708128462521791359286285169)


In [7]:
# Digital signature example
from hashlib import sha256

def sign(message: bytes, sk: tuple[int, int]):
    #  Hash the message
    h = sha256(message).hexdigest()
    # Convert hash to integer
    m = int(h, 16)
    # Sign the hash with private key
    s = pow(m, sk[0], sk[1])

    return s

def verify(message: bytes, signature: int, pk: tuple[int, int]):
    # Hash the message
    h = sha256(message).hexdigest()
    # Convert hash to integer
    m = int(h, 16)
    # Verify the signature
    s = pow(signature, pk[0], pk[1])
    return s == m

data = b'This is a super secret secret document.'
doc = {
    'data': data,
    'signature': sign(data, sk),
    'pk': pk,
}

verify(doc['data'], doc['signature'], doc['pk'])

True

# Certificate chain example

`openssl s_client -connect example.com:443 -showcerts`